# ⚙️ Notebook 2: Preprocessing

##  Data Cleaning
- Removed irrelevant columns (e.g., player_url, photo_url, club_logo).
- Handled missing values by:
  - Dropping rows with crucial missing stats
  - Filling minor missing values using mean/median

## ✨ Standardization
To make features comparable:
- Applied **MinMaxScaler** or **StandardScaler** to numerical attributes

## 🌀 Data Integration
Merged 6 CSV files into one DataFrame:
- Introduced a new column: `year`
- Unified all data into a long-format structure

## ✏️ Feature Selection
Selected key numerical attributes for analysis:
- Overall, Potential, Pace, Shooting, Passing, Dribbling, Defending, Physical, Value, Age

## 👥 Label Encoding (For LDA)
- Used `player_positions` or `best_position` as class labels for supervised learning algorithms (e.g., LDA).

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
import pandas as pd
import numpy as np

# Update this if your folder name is different
data_path = '/content/drive/MyDrive/Colab Notebooks/Football Dimensionality Reduction Project/data/'
csv_files = sorted([f for f in os.listdir(data_path) if f.endswith('.csv')])


Mounted at /content/drive


In [ ]:
dfs = {}
for file in csv_files:
    year = file.split('_')[1].split('.')[0]
    df = pd.read_csv(os.path.join(data_path, file))
    dfs[year] = df
    print(f"FIFA {year} → {df.shape}")


FIFA 15 → (15465, 104)
FIFA 16 → (14881, 104)
FIFA 17 → (17009, 104)
FIFA 18 → (17592, 104)
FIFA 19 → (17770, 104)
FIFA 20 → (18278, 104)


In [ ]:
# Identify features that exist in all datasets (intersection)
common_cols = set(dfs['15'].columns)
for df in dfs.values():
    common_cols &= set(df.columns)

# From those, select relevant features
key_cols = ['short_name', 'age', 'height_cm', 'weight_kg', 'overall', 'potential',
            'value_eur', 'wage_eur', 'international_reputation', 'skill_moves',
            'weak_foot', 'work_rate', 'team_position', 'pace', 'shooting',
            'passing', 'dribbling', 'defending', 'physic']

# Reduce all datasets to these features
for year in dfs:
    dfs[year] = dfs[year][[col for col in key_cols if col in dfs[year].columns]]
    dfs[year]['year'] = int('20' + year)


<ipython-input-3-591bb33f14f6>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfs[year]['year'] = int('20' + year)


In [ ]:
df_all = pd.concat(dfs.values(), ignore_index=True)
print("Final combined shape:", df_all.shape)


Final combined shape: (100995, 20)


In [ ]:
df_all = df_all.dropna(thresh=int(0.8 * df_all.shape[1]))  # Keep rows with ≥80% data
df_all = df_all.dropna()  # Drop remaining rows with any missing value


In [ ]:
# Label encode 'work_rate' and 'team_position'
from sklearn.preprocessing import LabelEncoder

cat_cols = ['work_rate', 'team_position']
encoders = {}

for col in cat_cols:
    enc = LabelEncoder()
    df_all[col] = enc.fit_transform(df_all[col])
    encoders[col] = enc


In [ ]:
from sklearn.preprocessing import StandardScaler

features = [col for col in df_all.columns if col not in ['short_name', 'year']]
X = df_all[features]
X_scaled = StandardScaler().fit_transform(X)

# Save X_scaled and player names for future notebooks
df_processed = pd.DataFrame(X_scaled, columns=features)
df_processed['short_name'] = df_all['short_name'].values
df_processed['year'] = df_all['year'].values

df_processed.to_csv('/content/drive/MyDrive/Colab Notebooks/Football Dimensionality Reduction Project/data/processed_fifa.csv', index=False)
print("✅ Preprocessing complete. Saved to processed_fifa.csv")


✅ Preprocessing complete. Saved to processed_fifa.csv
